In [4]:
import numpy as np
import os
import skimage
import skimage.io

## Reading patches

In [9]:
train_dir_1 = 'patches/train/1'
train_dir_0 = 'patches/train/0'

def read_images_flat(dir_name):
    for im_file in os.listdir(dir_name):
        im = skimage.io.imread(os.path.join(dir_name, im_file), as_grey=False)
        yield 1. - im.flatten() / 255.
    
    
X_train_0 = list(read_images_flat('patches/train/0'))
X_train_1 = list(read_images_flat('patches/train/1'))

X_train = X_train_0 + X_train_1
y_train = [0] * len(X_train_0) + [1] * len(X_train_1)

X_valid_0 = list(read_images_flat('patches/val/0'))
X_valid_1 = list(read_images_flat('patches/val/1'))

X_valid = X_valid_0 + X_valid_1
y_valid = [0] * len(X_valid_0) + [1] * len(X_valid_1)

## Models quality

In [20]:
import sklearn
import sklearn.neighbors
import sklearn.metrics
import sklearn.linear_model

### KNN

In [13]:
model = sklearn.neighbors.KNeighborsClassifier(
    algorithm='auto', leaf_size=30, metric='minkowski',
    metric_params=None, n_jobs=4, n_neighbors=10, p=2,
    weights='uniform')

model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=4, n_neighbors=10, p=2,
           weights='uniform')

In [21]:
y_valid_pred = model.predict(X_valid)
y_valid_pred_p = model.predict_proba(X_valid)[:,1]

print sklearn.metrics.classification_report(y_valid, y_valid_pred)
print 'AUC', sklearn.metrics.roc_auc_score(y_valid, y_valid_pred_p)

             precision    recall  f1-score   support

          0       0.74      1.00      0.85      2381
          1       0.99      0.30      0.46      1186

avg / total       0.82      0.77      0.72      3567

AUC 0.910162167752


### LogisticRegression

In [23]:
model = sklearn.linear_model.LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [24]:
y_valid_pred = model.predict(X_valid)
y_valid_pred_p = model.predict_proba(X_valid)[:,1]

print sklearn.metrics.classification_report(y_valid, y_valid_pred)
print 'AUC', sklearn.metrics.roc_auc_score(y_valid, y_valid_pred_p)

             precision    recall  f1-score   support

          0       0.96      0.96      0.96      2381
          1       0.91      0.92      0.92      1186

avg / total       0.94      0.94      0.94      3567

AUC 0.983639450314
